[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vanguardfox/ads509-group5/blob/main/ADS509_Final_Project.ipynb)

# Evaluating Financial Independence / Retire Early (FIRE) Thresholds by Demographic

What is the Financial Independence number the FIRE community is aiming for by age, location, and over time?

# Group 5 Objective:
 The purpose of this project is to evaluate the FIRE financial thresholds by demographic groups such as age and location as well as changes in trends over times. Resulting analysis can be used in a variety of applications including leading economic sentiment predictions, generational assessment of financial health, and others via a Flask implementation.

# Reddit Scrape

Community: r/FIRE \
thread: /What_is_your_fire_number/ \
Code Reference: with help from ChatGPT https://chatgpt.com/ and https://colab.research.google.com/


**Privacy note:**

*   Credentials were created for the class but might have to figure out how to hide
*   Username will also have to be masked to be anonymous

In [1]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

(Optional: Install packages as needed)

In [ ]:
#!pip install requests beautifulsoup4

Import Libraries

In [2]:
# Library
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
import csv
import requests
from typing import Optional, Tuple
import time
import datetime
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

from nltk.util import pairwise
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth
from os import read
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

## Initialize API Credentials

In [3]:
# Reddit API credentials
client_id = '8-xAlbBmc1_FYraSwJENtw'
client_secret = 'Jg9JyK9R_ixyNESMaGE3BwLzmBhf2w'
username = 'Western_Signal4499'
password = 'ADS509finAl'
user_agent = 'ADS509Scrapper/0.1 by Western_Signal4499'

# Get the OAuth token
auth = HTTPBasicAuth(client_id, client_secret)
headers = {'User-Agent': user_agent}
data = {'grant_type': 'password', 'username': username, 'password': password}

response = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, headers=headers, data=data)

if response.status_code == 200:
    token = response.json()['access_token']
else:
    print(f"Error: {response.status_code}, {response.text}")

## Use access_token field from JSON to query threads

In [4]:
# Use the token to search for threads
headers['Authorization'] = f'bearer {token}'

# Constructs a string for the r/FIRE URL to query "what is your fire number"
# restrict_sr=1 ensures that the query is constrained to only this subreddit
search_url = "https://oauth.reddit.com/r/FIRE/search?q=what%20is%20your%20fire%20number&restrict_sr=1"

# Store results of GET requrest to the API for the given "search_url"
search_response = requests.get(search_url, headers=headers)

# If the API GET is successful...
if search_response.status_code == 200: # Code 200 means the request was flagged as successful
    search_data = search_response.json() # Parse the JSON data from the response
    #print(search_data)
    threads = search_data['data']['children'] # Extract the list of threads

    # Initialize a CSV and writes an head with columns (within .writerow() arg)
    with open('fire_comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Thread ID', 'Title', 'User', 'Comment', 'Post Time', 'Comment Time'])

        # Iterate through each thread from API's response JSON
        for thread in threads:
            thread_title = thread['data']['title'] # Parse and extract title
            thread_id = thread['data']['id'] # Parse and extract id
            thread_time = thread['data']['created_utc']  # Extract post time (in Unix time)
            # Convert the post time to a parsible date format
            thread_time_norm = datetime.datetime.utcfromtimestamp(thread_time).strftime('%Y-%m-%d %H:%M:%S')

            # Prepare URL as a string for better code handling
            comments_url = f"https://oauth.reddit.com/r/FIRE/comments/{thread_id}.json"

            # Store results of GET to fetch comments of the current iterated thread
            comments_response = requests.get(comments_url, headers=headers)
            if comments_response.status_code == 200: # Again, check if GET is successful
                comments_data = comments_response.json() # Store JSON
                comments = comments_data[1]['data']['children'] # Parse and extract comments

                # Iterate through each comment
                for comment in comments:
                    if comment['kind'] == 't1': # Only iterate on top comments
                        username = comment['data']['author'] # Extract username
                        body = comment['data']['body'] # Extract text body

                        # Extract comment time, convert
                        comment_time = comment['data']['created_utc']
                        comment_time_norm = datetime.datetime.utcfromtimestamp(comment_time).strftime('%Y-%m-%d %H:%M:%S')

                        # Write parsed data into respective CSV row, mapped to column
                        writer.writerow([thread_id,
                                         thread_title,
                                         username,
                                         body,
                                         thread_time_norm,
                                         comment_time_norm])

                print(f"Thread ID: {thread_id}, Title: {thread_title}, User1: {username}, Comment: {body}, Post Time: {thread_time_norm}, Comment Time: {comment_time_norm}")

            else:
                print(f"Failed to retrieve comments for thread ID {thread_id} with status code {comments_response.status_code}")

else:
    print(f"Failed to retrieve search results with status code {search_response.status_code}")

Thread ID: 1fqzb0p, Title: What is your fire number? , User1: muy_carona, Comment: $8,675,309, Post Time: 2024-09-27 22:00:15, Comment Time: 2024-09-27 23:21:33
Thread ID: rfxxkz, Title: What is your fire number?, User1: xmxprztm, Comment: 4M at least. Going to move to Spain, Post Time: 2021-12-14 03:24:03, Comment Time: 2021-12-14 16:43:40
Thread ID: zddhma, Title: For folks who plan to FIRE in HCOL / VHCOL areas, what is your FIRE number? Why not move to MCOL / LCOL area?, User1: [deleted], Comment: Some services or experiences are only available in hcol areas. Good fine dining options and/or weather and or ___., Post Time: 2022-12-05 17:36:13, Comment Time: 2022-12-27 18:53:13
Thread ID: 10ex17r, Title: What is your FIRE number?, User1: zenjdg, Comment: My ideal scenario is $100k per month in tax-free cash flow., Post Time: 2023-01-18 03:44:27, Comment Time: 2023-11-04 15:20:39
Thread ID: z5a29v, Title: What is your fire number and how did you arrive at it?, User1: [deleted], Commen

## Verify the contents of the CSV that was created above and store as DataFrame

In [5]:
rfire_df = pd.read_csv('fire_comments.csv')
rfire_df.head()

,Thread ID,Title,User,Comment,Post Time,Comment Time
0,1fqzb0p,What is your fire number?,manimopo,1.5m with paid off houses,2024-09-27 22:00:15,2024-09-27 22:14:27
1,1fqzb0p,What is your fire number?,Animag771,I'm just waiting for someone to say something ...,2024-09-27 22:00:15,2024-09-27 22:43:43
2,1fqzb0p,What is your fire number?,wowsocool4u,$1.9m. Targeting a few months after i turn 50...,2024-09-27 22:00:15,2024-09-27 22:24:04
3,1fqzb0p,What is your fire number?,Aromatic_Heart,"On days I hate my job, 600k. The other days, d...",2024-09-27 22:00:15,2024-09-28 03:15:50
4,1fqzb0p,What is your fire number?,Ziqach,$2M with a paid off house. So stressful to see...,2024-09-27 22:00:15,2024-09-27 22:59:17


# Data Preprocessing

## Preliminary dataframe

In [6]:
rfire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Thread ID     1460 non-null   object
 1   Title         1460 non-null   object
 2   User          1460 non-null   object
 3   Comment       1460 non-null   object
 4   Post Time     1460 non-null   object
 5   Comment Time  1460 non-null   object
dtypes: object(6)
memory usage: 68.6+ KB


In [7]:
rfire_df['Title'].unique()

array(['What is your fire number? ', 'What is your fire number?',
       'For folks who plan to FIRE in HCOL / VHCOL areas, what is your FIRE number? Why not move to MCOL / LCOL area?',
       'What is your FIRE number?',
       'What is your fire number and how did you arrive at it?',
       "How to split finances when one is FIRE'd and one isn't FIRE'ing",
       'What is your FIRE number and at what age do you want to achieve it by?',
       'We’re all privileged',
       'How my wife and I will retire before 50; neither of us make over $75K',
       '10 tips from my 20 year journey to financial freedom',
       'Is toxic corporate culture why most of us want to Fire? ',
       'The cliff is steep: How do you choose to retire early when working 5 more years can bring so much more',
       'How did you come up with your FIRE number? What is the best calculator, way to get at it?',
       'Forgive me if this is too personal, but what was your estimated net worth around 18 and how did 

## Clean Dataset

In [136]:
# Retrive titles that contain the words "what is your fire number"
fire_df = rfire_df[rfire_df['Title'].str.contains(r'what\s*is\s*your\s*fire\s*number', case=False, na=False)]

print(fire_df['Title'].unique())
print(fire_df.info())

['What is your fire number? ' 'What is your fire number?'
 'For folks who plan to FIRE in HCOL / VHCOL areas, what is your FIRE number? Why not move to MCOL / LCOL area?'
 'What is your FIRE number?'
 'What is your fire number and how did you arrive at it?'
 'What is your FIRE number and at what age do you want to achieve it by?'
 'Married couples with young kids, what is your FIRE number and how is your progress?']
<class 'pandas.core.frame.DataFrame'>
Index: 432 entries, 0 to 1226
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Thread ID     432 non-null    object
 1   Title         432 non-null    object
 2   User          432 non-null    object
 3   Comment       432 non-null    object
 4   Post Time     432 non-null    object
 5   Comment Time  432 non-null    object
dtypes: object(6)
memory usage: 23.6+ KB
None


### Fire Number Extraction

Regular Expression Description

re.sub(r'[^a-zA-Z0-9\s$€£¥.]', '', text)

replaces matching substrings with an empty '' (effectively removing it)

Breakdown: \
^a-zA-Z0-9 = NOT alphanumeric characters \
\s = whitespace \
$€£¥ = Currency symbols \
. = Period / Decimal \

Objective:  \
Regular Expression removes everything that isn't an alphanumeric character or symbol above AFTER we've already removed periods from the line

text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

Breakdown: \
(?<!\d) Only match if the character before the period is not a digit (\d) *only checking with ?*


In [137]:
# Define comment cleaning steps
def clean_and_extract(text: str) -> str:
    """
    Cleans the input text by:
    - Removing URLs
    - Keep alphanumeric characters, spaces, dashes, and currency symbols
    - Normalizing financial identifiers
    - Converting text to lowercase
    - Removing isolated periods

    Args:
        text (str): The input text to be cleaned.

    Returns:
        str: The cleaned and normalized text.
    """
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Keep alphanumeric characters, spaces, slash, and currency symbols
    text = re.sub(r'[^a-zA-Z0-9\s$€£¥./-]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove isolated periods not part of numbers
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # Normalize financial identifiers
    financial_patterns = [
    # Pattern to match "Xmil/Yk" or "Xmil/Ymil" and replace it appropriately
    (re.compile(r'(?<!\w)\$?(\d+(?:\.\d+)?)\s*(?:mm|m|mil)\s*/\s*(?<!\w)\$?(\d+(?:\.\d+)?)\s*(k|mm|m|mil)(?=\s|$)', re.IGNORECASE), r'\1 million/\2 \3'),

    # Pattern to match single instances of millions (e.g., "1.5mm", "5mil") and replace with "X million"
    (re.compile(r'(?<!\w)\$?(\d+(?:\.\d+)?)\s*(?:mm|m|mil)(?=\s|$)', re.IGNORECASE), r'\1 million '),

    # Pattern to match "Xb" and replace with "X billion"
    (re.compile(r'(?<!\w)\$?(\d+)\s*b(?=\s|$)', re.IGNORECASE), r'\1 billion '),

    # Pattern to match "X mil" or "X million" and replace with "X million"
    (re.compile(r'(?<!\w)\$?(\d+)\s*(mil|million)(?=\s|$)', re.IGNORECASE), r'\1 million '),

    # Pattern to match "Xk" or "X thousand" and replace with "X thousand"
    (re.compile(r'(?<!\w)\$?(\d+)\s*(k|thousand)(?=\s|$)', re.IGNORECASE), r'\1 thousand ')
    ]


    for pattern, replacement in financial_patterns:
        text = pattern.sub(replacement, text)

    # Remove extra spaces after replacements
    text = re.sub(r'\s+', ' ', text).strip()

    return text


# Define extraction of fire number
def extract_fire_number(text: str) -> str:
    fire_number_match = re.search(r'(\d+(\.\d+)?)\s*(million|billion|thousand)', text)
    return fire_number_match.group(0) if fire_number_match else None

# Define conversion of fire number to numeric
def convert_fire_number_to_numeric(fire_number: str) -> float:
    if not fire_number:
        return 0.0

    # Define conversion factors
    conversion_factors = {
        'million': 1_000_000,
        'billion': 1_000_000_000,
        'thousand': 1_000
    }

    # Extract numeric part and the suffix
    match = re.match(r'(\d+(\.\d+)?)\s*(million|billion|thousand)', fire_number)
    if match:
        value = float(match.group(1))
        suffix = match.group(3)
        return value * conversion_factors[suffix]

    return 0.0

In [139]:
# Apply the cleaning function to the 'Comment' column
fire_df['Cleaned Comment'] = fire_df['Comment'].apply(clean_and_extract)

# Extract FIRE number
fire_df['FIRE #'] = fire_df['Cleaned Comment'].apply(extract_fire_number)

# Convert FIRE Number from string to numerical value
fire_df['FIRE Number'] = fire_df['FIRE #'].apply(convert_fire_number_to_numeric)

# Drop the original FIRE Number column if no longer needed
fire_df.drop(columns=['FIRE #'], inplace=True)

In [144]:
# Check for regular expression conversion
random_comments = fire_df[['Comment','Cleaned Comment', 'FIRE Number']].sample(n=10, random_state=10)  # Set random_state for reproducibility
print(random_comments)

                                                Comment  \
65    600k is where I don't worry about expenses. 1....   
266   Could be there right now if I wanted to.\n\nMy...   
188                 4M at least. Going to move to Spain   
47                                           1.2, rent.   
287                             3.6 with mortgage paid.   
1223  Marrying whoever after my Bar Exam this Novemb...   
279   7mil. Married w/ 2 kids… 33yo and 30yo. Goal i...   
338   What I want to pull out my first year x28.5 in...   
163                                           2 - 3 Mil   
357   $2,000,000 gives us $80,000/year. That’s what ...   

                                        Cleaned Comment  FIRE Number  
65    600 thousand is where i dont worry about expen...     600000.0  
266   could be there right now if i wanted to my goa...    2000000.0  
188           4 million at least going to move to spain    4000000.0  
47                                             1.2 rent          0

In [141]:
# Display the first few rows of the DataFrame
print("FIRE Dataframe\n")
print(fire_df.head(),"\n")

# Print general information about the DataFrame with a proper newline
print("FIRE DataFrame General Info\n")
print(fire_df.info())

FIRE Dataframe

  Thread ID                       Title            User  \
0   1fqzb0p  What is your fire number?         manimopo   
1   1fqzb0p  What is your fire number?        Animag771   
2   1fqzb0p  What is your fire number?      wowsocool4u   
3   1fqzb0p  What is your fire number?   Aromatic_Heart   
4   1fqzb0p  What is your fire number?           Ziqach   

                                             Comment            Post Time  \
0                          1.5m with paid off houses  2024-09-27 22:00:15   
1  I'm just waiting for someone to say something ...  2024-09-27 22:00:15   
2  $1.9m.  Targeting a few months after i turn 50...  2024-09-27 22:00:15   
3  On days I hate my job, 600k. The other days, d...  2024-09-27 22:00:15   
4  $2M with a paid off house. So stressful to see...  2024-09-27 22:00:15   

          Comment Time                                    Cleaned Comment  \
0  2024-09-27 22:14:27                   1.5 million with paid off houses   
1  2024-09-2

### Create CountVectorizer matrix for the cleaned comments

In [ ]:
vectorizer_ct = CountVectorizer(stop_words='english')
X_ct = vectorizer_ct.fit_transform(fire_df['Cleaned Comment'])

### Create Term Frequency - Inverse Document Frequency scored vectors

In [ ]:
vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_features=1000) # Cap the size for memory
X_tfidf = vectorizer_tfidf.fit_transform(fire_df['Cleaned Comment'])

## Exploratory Data Analysis

# Model Development

## Topic Modeling

### Latent Dirichlet Allocation

In [ ]:
lda = LatentDirichletAllocation(n_components=5, random_state=509)
lda.fit(X_ct)

LatentDirichletAllocation(n_components=5, random_state=509)

#### Examine topics from LDA at n=5 topics (arbritrary)

In [ ]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx}:")

        #For each topic, go through index i and retrieve word from that location
        #Topic is a vector of weights of each word for a given topic (higher = more important)
        #argsort() return indices of elements in topic in ascending importance
        #:-num_top_words - 1 = Start from last element -1 and move back num_top_words positions
        # --- This is to remove the lesser important words for interpretation purposes
        # -1 reverse order so it's descending now that noisy words are removed.
        print(" ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [ ]:
feature_names_lda = vectorizer_ct.get_feature_names_out()
num_top_words = 10
display_topics(lda, feature_names_lda, num_top_words)

Topic 0:
want like money just youre time way income expenses things
Topic 1:
money year years work dont time retirement retire just job
Topic 2:
house years 5m im paid million number want kids 2m
Topic 3:
work im just people age make time number job dont
Topic 4:
years work life time dont like im just year income


#### Result
Selecting n_components=5 might not be the correct number of topics to be able to clearly separate words into themes. Need to try another topic number.

### Retry LDA with n_components=8

In [ ]:
lda = LatentDirichletAllocation(n_components=8, random_state=509)
lda.fit(X_ct)
feature_names_lda = vectorizer_ct.get_feature_names_out()
num_top_words = 10
display_topics(lda, feature_names_lda, num_top_words)

Topic 0:
just like want money im time way number youre life
Topic 1:
money year years retirement life retire deleted kids like work
Topic 2:
want work im years live 5m area time dont paid
Topic 3:
work im million age target number just house paid ill
Topic 4:
years life work dont like just money im year people
Topic 5:
lifestyle dont partner money youre time just need want work
Topic 6:
years im time want house work retire 5m kids money
Topic 7:
retirement income years year real cash estate really market number


## DEPENDENCY.
Need to find a way to import a date of post or age of post so the model below can extract some sort of time or year for a time-series chart. Else, will have to select a different groupby.

### Sentiment Analysis with VaderSentiment

Reference

Hutto, C. J., Klein, E., Pantone, P., Berry, G., & Suresh, M. (2024, August 19). Documentation source code for nltk.sentiment.vader. https://www.nltk.org/_modules/nltk/sentiment/vader.html

In [ ]:
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/johnvincent/nltk_data...


True

In [ ]:
analyzer = SentimentIntensityAnalyzer() # Intialize the analyzer as object

In [ ]:
def get_sentiment_score(comment):
    scores = analyzer.polarity_scores(comment)
    return pd.Series([scores['neg'], scores['neu'], scores['pos'], scores['compound']],
                    index=['Negative', 'Neutral', 'Positive', 'Compound'])

In [ ]:
rfire_df[['Negative', 'Neutral', 'Positive', 'Compound']] = rfire_df['clean_comment'].apply(get_sentiment_score)

In [ ]:
rfire_df[['clean_comment', 'Negative', 'Neutral', 'Positive', 'Compound']].head(10)

,clean_comment,Negative,Neutral,Positive,Compound
0,1.5m with paid off houses,0.000,1.000,0.000,0.0000
1,im just waiting for someone to say something l...,0.172,0.828,0.000,-0.4019
2,$1.9m targeting a few months after i turn 50 ...,0.074,0.926,0.000,-0.1531
3,on days i hate my job 600k the other days depe...,0.171,0.829,0.000,-0.5719
4,$2m with a paid off house so stressful to see ...,0.233,0.767,0.000,-0.6418
5,our target back in 2014 was $1.2m and a paidof...,0.060,0.870,0.070,-0.3400
6,500k give or take but i am not retiring in th...,0.000,1.000,0.000,0.0000
7,i started at 1.5m 1015 years ago now being 5 ...,0.060,0.783,0.157,0.4019
8,re ship sailed but still working on fi house i...,0.000,0.906,0.094,0.6124
9,funny to read all these numbers while the medi...,0.000,0.661,0.339,0.7717


### OPTIONAL:GROUPBY ANALYSIS, which requires additional JSON features to be pulled into CSV

In [ ]:
sentiment_by_<FEATURE NAME> = df.groupby('<INSERT FEATURE NAME>'[['negative', 'neutral', 'positive', 'compound']].mean()
sentiment_by_<FEATURE NAME>

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(sentiment_by_year.index, sentiment_by_year['Compound'], marker='o')
plt.title('Sentiment Analysis Over Time')
plt.xlabel('<FEATURE>')
plt.ylabel('Compound Sentiment Score')
plt.grid(True)
plt.legend()
plt.show()

### Latent Semantic Analysis

In [ ]:
n_topics = 5
lsa = TruncatedSVD(n_components=n_topics, random_state=509) # Initialize model
lsa_topic_matrix = lsa.fit_transform(X_tfidf) # Fit and transform for latent topics

In [ ]:
feature_names_tfidf = vectorizer_tfidf.get_feature_names_out()

In [ ]:
display_topics(lsa, feature_names_tfidf, num_top_words) # Show top 10 words for each topic

Topic 0:
years im work time want money just dont kids like
Topic 1:
deleted post waiting start tax lower earlier regular numbers portfolio
Topic 2:
5m house paid million target liquid 2m mil 3m years
Topic 3:
million 10 years retire 12 22 ill worth near 45
Topic 4:
house paid time kids money want mil live people work


#### Show results
Row = score representing relevance of topic to the clean_comment

In [ ]:
topics = pd.DataFrame(lsa_topic_matrix, columns=[f'Topic {i}' for i in range(n_topics)])
comb_topics = pd.concat([rfire_df, topics], axis=1)
comb_topics[['clean_comment', 'Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4']].head(10)

,clean_comment,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
0,1.5m with paid off houses,0.109908,-0.000592,0.369511,-0.216351,0.116825
1,im just waiting for someone to say something l...,0.124805,0.006525,-0.033491,0.019276,-0.024431
2,$1.9m targeting a few months after i turn 50 ...,0.154432,-0.000637,0.116150,-0.031073,0.100695
3,on days i hate my job 600k the other days depe...,0.129950,-0.000498,0.017841,0.218858,0.019035
4,$2m with a paid off house so stressful to see ...,0.149271,-0.000604,0.340298,-0.079957,0.321560
5,our target back in 2014 was $1.2m and a paidof...,0.200973,-0.000629,0.186576,-0.057527,0.029379
6,500k give or take but i am not retiring in th...,0.037700,-0.000117,0.017364,0.015099,-0.023987
7,i started at 1.5m 1015 years ago now being 5 ...,0.203734,-0.000807,0.302675,-0.223604,-0.129272
8,re ship sailed but still working on fi house i...,0.189211,-0.000571,0.143160,-0.014286,-0.018408
9,funny to read all these numbers while the medi...,0.098286,-0.000036,0.012844,0.023013,-0.087721


# Build Flask Application

## Set-up Flask


Import Packages

In [ ]:
# Import packages
#!pip install Flask
#!pip install pyngrok

Upload Libraries

In [ ]:
# Libraries:
from flask import Flask, request, jsonify
import threading
from pyngrok import ngrok

## Simple Flask Application

In [ ]:
# Set up ngrok authtoken
!ngrok authtoken 2mxORvZevUlWiZcP42H0kKWt5Db_62uzFBWQsdosv7Eau5TWZ

# Create Flask app
app = Flask(__name__)

@app.route('/')
def home():
    return "Welcome to the FIRE Number Analyzer!"

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.json
    # Placeholder for analysis logic
    return jsonify({"message": "Analysis complete!"})

# Function to run the Flask app
def run_app():
    app.run(port=5000)

# Start Flask app in a new thread
threading.Thread(target=run_app).start()

# Set up a tunnel to the Flask app
public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * ngrok tunnel "NgrokTunnel: "https://6fb1-35-236-152-11.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
